# Model: WangchanBERTa

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import numpy as np
import math
import re
import torch
import json
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

# Raw Data

In [2]:
df = pd.read_csv('../datasets/tscc_v0.1-judgement.csv')
print(len(df))
df.head()

1207


,issueid,dekaid,year,category,issueno,lawids,fact,decision,isact,isexternalelements,isinternalelement,isintent,isneglect,iscause,isjustify,isexcuse,isguilty,isattempt,isattemptimpossible
0,1,1478/2528,2528,LB,1,"CC-288-00,CC-083-00,CC-063-00",จำเลยกับพวกร่วมกันใช้อาวุธปืนยิงผู้ตายถูกที่ด้...,จำเลยจึงมีความผิดฐานฆ่าผู้ตายโดยเจตนา,1,1,1,1,-1,1,0,0,1,0,-1
1,2,1548/2531,2531,LB,1,CC-288-00,จำเลยที่ 1 ซึ่งเคยมีเรื่องทะเลาะกับผู้ตายมาก่อ...,จำเลยที่ 1 จึงมีความผิดฐานฆ่าผู้ตายโดยเจตนา,1,1,1,1,-1,1,0,0,1,0,-1
2,3,1548/2531,2531,LB,2,"CC-290-00,CC-083-00",ส่วนจำเลยที่ 2 ที่ 3 และที่ 4 นั้น ได้ความว่าก...,การที่จำเลยที่ 1 ใช้เหล็กแหลมแทงผู้ตายโดยเจตนา...,1,1,1,1,-1,1,0,0,1,0,-1
3,4,1548/2531,2531,LB,3,"CC-288-00,CC-083-00",ส่วนจำเลยที่ 2 ที่ 3 และที่ 4 นั้น ได้ความว่าก...,การที่จำเลยที่ 1 ใช้เหล็กแหลมแทงผู้ตายโดยเจตนา...,0,-1,-1,-1,-1,-1,-1,-1,0,-1,-1
4,5,1697/2522,2522,LB,1,"CC-288-00,CC-083-00",โจทก์บรรยายฟ้องว่า จำเลยกับพวกที่ยังไม่ได้ตัวม...,จึงเป็นการกระทำโดยมีเจตนาฆ่าผู้ตาย แม้ข้อเท็จจ...,1,1,1,1,-1,1,0,0,1,0,-1


# Preprocess

## Filter non-relevant article ids (i.e. id <= 106)

In [3]:
lawids_df = df['lawids']

lawids = []
for ids in lawids_df:
    lawids.extend(ids.split(','))
lawids_set = set(lawids)
print('# Article IDs (before filtering out): ', len(lawids_set))

''' remove any articles with id <= 106 '''
labels = list(filter(lambda x: int(x.split('-')[1][:3]) > 106, lawids_set))
print('# Article IDs (after filtering out): ', len(labels))
labels


# Article IDs (before filtering out):  76
# Article IDs (after filtering out):  50


['CC-337-00',
 'CC-393-01',
 'CC-335bis-00',
 'CC-290-00',
 'CC-338-00',
 'CC-342-00',
 'CC-343-00',
 'CC-340-01',
 'CC-393-00',
 'CC-391-00',
 'CC-289(3)-00',
 'CC-339-00',
 'CC-328-01',
 'CC-289(4)-00',
 'CC-326-01',
 'CC-358-00',
 'CC-328-00',
 'CC-331-00',
 'CC-339-02',
 'CC-390-00',
 'CC-335bis-01',
 'CC-336-00',
 'CC-360-00',
 'CC-297-00',
 'CC-335-00',
 'CC-335-02',
 'CC-336bis-00',
 'CC-289(5)-00',
 'CC-326-00',
 'CC-328-02',
 'CC-296-00',
 'CC-340ter-00',
 'CC-340-00',
 'CC-291-00',
 'CC-326-02',
 'CC-298-00',
 'CC-354-00',
 'CC-330-00',
 'CC-329-00',
 'CC-289(2)-00',
 'CC-341-00',
 'CC-300-00',
 'CC-334-00',
 'CC-289(7)-00',
 'CC-352-00',
 'CC-289(6)-00',
 'CC-335-01',
 'CC-288-00',
 'CC-295-00',
 'CC-339-01']

## Filter \<discr>...\</discr> portion out of fact description

In [4]:
pattern = r'<discr>.*?</discr>'
df['filtered_fact'] = df.fact.apply(lambda x: re.sub(pattern, '', x)).copy()
df.head()

,issueid,dekaid,year,category,issueno,lawids,fact,decision,isact,isexternalelements,isinternalelement,isintent,isneglect,iscause,isjustify,isexcuse,isguilty,isattempt,isattemptimpossible,filtered_fact
0,1,1478/2528,2528,LB,1,"CC-288-00,CC-083-00,CC-063-00",จำเลยกับพวกร่วมกันใช้อาวุธปืนยิงผู้ตายถูกที่ด้...,จำเลยจึงมีความผิดฐานฆ่าผู้ตายโดยเจตนา,1,1,1,1,-1,1,0,0,1,0,-1,จำเลยกับพวกร่วมกันใช้อาวุธปืนยิงผู้ตายถูกที่ด้...
1,2,1548/2531,2531,LB,1,CC-288-00,จำเลยที่ 1 ซึ่งเคยมีเรื่องทะเลาะกับผู้ตายมาก่อ...,จำเลยที่ 1 จึงมีความผิดฐานฆ่าผู้ตายโดยเจตนา,1,1,1,1,-1,1,0,0,1,0,-1,จำเลยที่ 1 ซึ่งเคยมีเรื่องทะเลาะกับผู้ตายมาก่อ...
2,3,1548/2531,2531,LB,2,"CC-290-00,CC-083-00",ส่วนจำเลยที่ 2 ที่ 3 และที่ 4 นั้น ได้ความว่าก...,การที่จำเลยที่ 1 ใช้เหล็กแหลมแทงผู้ตายโดยเจตนา...,1,1,1,1,-1,1,0,0,1,0,-1,ส่วนจำเลยที่ 2 ที่ 3 และที่ 4 นั้น ได้ความว่าก...
3,4,1548/2531,2531,LB,3,"CC-288-00,CC-083-00",ส่วนจำเลยที่ 2 ที่ 3 และที่ 4 นั้น ได้ความว่าก...,การที่จำเลยที่ 1 ใช้เหล็กแหลมแทงผู้ตายโดยเจตนา...,0,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,ส่วนจำเลยที่ 2 ที่ 3 และที่ 4 นั้น ได้ความว่าก...
4,5,1697/2522,2522,LB,1,"CC-288-00,CC-083-00",โจทก์บรรยายฟ้องว่า จำเลยกับพวกที่ยังไม่ได้ตัวม...,จึงเป็นการกระทำโดยมีเจตนาฆ่าผู้ตาย แม้ข้อเท็จจ...,1,1,1,1,-1,1,0,0,1,0,-1,โจทก์บรรยายฟ้องว่า จำเลยกับพวกที่ยังไม่ได้ตัวม...


## One-hot Encoding

In [5]:
def label_encoding(case_lawids, lawid):
    if lawid in case_lawids:
        return 1
    else:
        return 0

# df2 = df.copy()
for label in labels:
    # df2[label] = df2.lawids.apply(lambda x: label_encoding(x, label))
    df[label] = df.lawids.apply(lambda x: label_encoding(x, label))
df.head()

,issueid,dekaid,year,category,issueno,lawids,fact,decision,isact,isexternalelements,...,CC-341-00,CC-300-00,CC-334-00,CC-289(7)-00,CC-352-00,CC-289(6)-00,CC-335-01,CC-288-00,CC-295-00,CC-339-01
0,1,1478/2528,2528,LB,1,"CC-288-00,CC-083-00,CC-063-00",จำเลยกับพวกร่วมกันใช้อาวุธปืนยิงผู้ตายถูกที่ด้...,จำเลยจึงมีความผิดฐานฆ่าผู้ตายโดยเจตนา,1,1,...,0,0,0,0,0,0,0,1,0,0
1,2,1548/2531,2531,LB,1,CC-288-00,จำเลยที่ 1 ซึ่งเคยมีเรื่องทะเลาะกับผู้ตายมาก่อ...,จำเลยที่ 1 จึงมีความผิดฐานฆ่าผู้ตายโดยเจตนา,1,1,...,0,0,0,0,0,0,0,1,0,0
2,3,1548/2531,2531,LB,2,"CC-290-00,CC-083-00",ส่วนจำเลยที่ 2 ที่ 3 และที่ 4 นั้น ได้ความว่าก...,การที่จำเลยที่ 1 ใช้เหล็กแหลมแทงผู้ตายโดยเจตนา...,1,1,...,0,0,0,0,0,0,0,0,0,0
3,4,1548/2531,2531,LB,3,"CC-288-00,CC-083-00",ส่วนจำเลยที่ 2 ที่ 3 และที่ 4 นั้น ได้ความว่าก...,การที่จำเลยที่ 1 ใช้เหล็กแหลมแทงผู้ตายโดยเจตนา...,0,-1,...,0,0,0,0,0,0,0,1,0,0
4,5,1697/2522,2522,LB,1,"CC-288-00,CC-083-00",โจทก์บรรยายฟ้องว่า จำเลยกับพวกที่ยังไม่ได้ตัวม...,จึงเป็นการกระทำโดยมีเจตนาฆ่าผู้ตาย แม้ข้อเท็จจ...,1,1,...,0,0,0,0,0,0,0,1,0,0


In [6]:
cols = df.columns
label_cols = list(cols[20:])
num_labels = len(label_cols)
print('num_labels: ', num_labels)
print('Label columns: ', label_cols)

num_labels:  50
Label columns:  ['CC-337-00', 'CC-393-01', 'CC-335bis-00', 'CC-290-00', 'CC-338-00', 'CC-342-00', 'CC-343-00', 'CC-340-01', 'CC-393-00', 'CC-391-00', 'CC-289(3)-00', 'CC-339-00', 'CC-328-01', 'CC-289(4)-00', 'CC-326-01', 'CC-358-00', 'CC-328-00', 'CC-331-00', 'CC-339-02', 'CC-390-00', 'CC-335bis-01', 'CC-336-00', 'CC-360-00', 'CC-297-00', 'CC-335-00', 'CC-335-02', 'CC-336bis-00', 'CC-289(5)-00', 'CC-326-00', 'CC-328-02', 'CC-296-00', 'CC-340ter-00', 'CC-340-00', 'CC-291-00', 'CC-326-02', 'CC-298-00', 'CC-354-00', 'CC-330-00', 'CC-329-00', 'CC-289(2)-00', 'CC-341-00', 'CC-300-00', 'CC-334-00', 'CC-289(7)-00', 'CC-352-00', 'CC-289(6)-00', 'CC-335-01', 'CC-288-00', 'CC-295-00', 'CC-339-01']


## Stats

In [7]:
print('Count of 1 per label: \n', df[label_cols].sum().sort_values(), '\n')
print('Count of 0 per label: \n', df[label_cols].eq(0).sum().sort_values())


Count of 1 per label: 
 CC-354-00         1
CC-289(3)-00      1
CC-298-00         1
CC-335bis-01      1
CC-335-01         2
CC-335bis-00      2
CC-338-00         2
CC-339-01         3
CC-342-00         3
CC-289(7)-00      3
CC-390-00         4
CC-340-00         5
CC-296-00         5
CC-343-00         6
CC-289(6)-00      6
CC-339-00         6
CC-360-00         7
CC-336-00         7
CC-337-00         8
CC-335-00         8
CC-358-00         8
CC-340-01         9
CC-340ter-00     10
CC-289(5)-00     10
CC-391-00        11
CC-336bis-00     11
CC-331-00        11
CC-300-00        13
CC-289(2)-00     13
CC-330-00        14
CC-339-02        15
CC-328-00        17
CC-291-00        18
CC-290-00        20
CC-393-00        20
CC-326-02        26
CC-297-00        26
CC-393-01        27
CC-328-02        40
CC-295-00        44
CC-335-02        46
CC-289(4)-00     48
CC-328-01        51
CC-326-01        57
CC-352-00        61
CC-326-00        61
CC-334-00        90
CC-329-00       108
CC-341-00       

## Multi one-hot labels

In [8]:
df['one_hot_labels'] = list(df[label_cols].values)
df.head()

,issueid,dekaid,year,category,issueno,lawids,fact,decision,isact,isexternalelements,...,CC-300-00,CC-334-00,CC-289(7)-00,CC-352-00,CC-289(6)-00,CC-335-01,CC-288-00,CC-295-00,CC-339-01,one_hot_labels
0,1,1478/2528,2528,LB,1,"CC-288-00,CC-083-00,CC-063-00",จำเลยกับพวกร่วมกันใช้อาวุธปืนยิงผู้ตายถูกที่ด้...,จำเลยจึงมีความผิดฐานฆ่าผู้ตายโดยเจตนา,1,1,...,0,0,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,1548/2531,2531,LB,1,CC-288-00,จำเลยที่ 1 ซึ่งเคยมีเรื่องทะเลาะกับผู้ตายมาก่อ...,จำเลยที่ 1 จึงมีความผิดฐานฆ่าผู้ตายโดยเจตนา,1,1,...,0,0,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,1548/2531,2531,LB,2,"CC-290-00,CC-083-00",ส่วนจำเลยที่ 2 ที่ 3 และที่ 4 นั้น ได้ความว่าก...,การที่จำเลยที่ 1 ใช้เหล็กแหลมแทงผู้ตายโดยเจตนา...,1,1,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,1548/2531,2531,LB,3,"CC-288-00,CC-083-00",ส่วนจำเลยที่ 2 ที่ 3 และที่ 4 นั้น ได้ความว่าก...,การที่จำเลยที่ 1 ใช้เหล็กแหลมแทงผู้ตายโดยเจตนา...,0,-1,...,0,0,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,1697/2522,2522,LB,1,"CC-288-00,CC-083-00",โจทก์บรรยายฟ้องว่า จำเลยกับพวกที่ยังไม่ได้ตัวม...,จึงเป็นการกระทำโดยมีเจตนาฆ่าผู้ตาย แม้ข้อเท็จจ...,1,1,...,0,0,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
labels = list(df.one_hot_labels.values)
facts = list(df.filtered_fact.values)
print(len(labels))
print(len(facts))


1207
1207


# Create Dataset

## Tokenizer from Model 

In [10]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"

In [11]:
max_length = 416
tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                revision='main',
                model_max_length=max_length,)

encodings = tokenizer(facts, max_length=max_length, padding=True, truncation=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

tokenizer outputs:  dict_keys(['input_ids', 'attention_mask'])


In [12]:
type(encodings)

transformers.tokenization_utils_base.BatchEncoding

In [13]:
input_ids = encodings['input_ids'] # tokenized and encoded sentences
attention_masks = encodings['attention_mask'] # attention masks

## Stratifying

In [14]:
label_counts = df.one_hot_labels.astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(df[df.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

df label indices with only one instance:  [707, 702, 410, 403, 397, 389, 316, 225, 219, 217, 82, 50]


In [15]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

## Train, Valid Split

In [23]:
# Use train_test_split to split our data into train and validation sets

train_inputs, valid_inputs, train_labels, \
    valid_labels, train_masks, valid_masks = train_test_split(input_ids, labels, attention_masks,
                                                            random_state=2020, test_size=0.10, stratify=labels)

# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.FloatTensor(train_labels)
train_masks = torch.tensor(train_masks)

valid_inputs = torch.tensor(valid_inputs)
valid_labels = torch.FloatTensor(valid_labels)
valid_masks = torch.tensor(valid_masks)

## Dataset

## Change to dictionary for Trainer class

In [41]:
class TSCCDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, masks, labels):
        self.inputs = inputs
        self.masks = masks
        self.labels = labels

    def __getitem__(self, idx):
        item = {'input_ids': self.inputs[idx], 'attention_mask': self.masks[idx], 'labels': self.labels[idx]}
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TSCCDataset(train_inputs, train_masks, train_labels)
valid_dataset = TSCCDataset(valid_inputs, valid_masks, valid_labels)

# Load Model & Set Params

In [42]:
#revision = "finetuned@wisesight_sentiment"
revision = None
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels, revision=revision, problem_type='multi_label_classification'
)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
'''Trying forward pass'''
outputs = model(input_ids=train_dataset[0]['input_ids'].unsqueeze(0), 
    labels=train_dataset[0]['labels'].unsqueeze(0),
    attention_mask=train_dataset[0]['attention_mask'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6905, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.3245,  0.0693, -0.0217, -0.3090, -0.1931,  0.0416,  0.1648,  0.1066,
         -0.0857, -0.1188, -0.3371, -0.1882, -0.2366,  0.0117, -0.0945,  0.1217,
         -0.3972, -0.1896, -0.1445,  0.0166,  0.3159, -0.1436,  0.3762, -0.0958,
         -0.0995,  0.2255, -0.2332, -0.3199,  0.1950,  0.2023,  0.3743,  0.3246,
         -0.0676, -0.1899,  0.3902,  0.2862, -0.0743, -0.0765,  0.0052, -0.5541,
         -0.0432, -0.3443, -0.6405,  0.0892, -0.4183,  0.3578,  0.5360,  0.0300,
          0.2378,  0.2647]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Train

In [44]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

In [45]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [46]:
batch_size = 32
metric_name = 'f1'

In [47]:
training_args = TrainingArguments(
    output_dir='./multilabel_classification_task',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=batch_size,  # batch size per device during training
    per_device_eval_batch_size=batch_size,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./multilabel_classification_task_logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #evaluation_strategy='epoch'
)

In [48]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,        # evaluation dataset
    compute_metrics=compute_metrics,
)

In [51]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [54]:
trainer.train()

  0%|          | 0/102 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 8.71 GB, other allocations: 9.59 GB, max allowed: 18.13 GB). Tried to allocate 253.50 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
result_eval = trainer.evaluate()
result_eval

In [ ]:
save_path = "./models/multilabel/1/"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# References

- WangchanBERTa Tutorial: https://colab.research.google.com/drive/1Kbk6sBspZLwcnOE61adAQo30xxqOQ9ko

- Slightly oudated multilabel classification using HuggingFace's Transformer: https://towardsdatascience.com/transformers-for-multilabel-classification-71a1a0daf5e1

- Another multilabel classification examples: https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb